Importing lib to get data in required format

In [1]:
import numpy as np #library to handle data in a vectorized manner
import pandas as pd #library for data analysis
import json #library for json file
!pip install geopy
from geopy.geocoders import Nominatim #convert address to longitude and latitude
import requests #library to handle requests
from pandas.io.json import json_normalize #convert json file to pandas data frame
#matplotlib plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans #for clustering
!conda install -c conda-forge folium=0.5.0 --yes 
import folium #library for plotting maps
print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
CLIENT_ID = 'your id' # your Foursquare ID
CLIENT_SECRET = 'your secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
 

In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


# SEARCH FOR HOTELS IN TORNOTO 

In [77]:
search_query='Hotel'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude,search_query,radius, LIMIT)
url 

'https://api.foursquare.com/v2/venues/search?&client_id=EOKYOPKM1KJJ301CDFXD1EZ1PBCZG0IQMJYZKANFDLHGKS5U&client_secret=H4BWIMNLNIJ14YONR3O0D2VWOJW0SXEE2MABSS5F2LDG5PW1&v=20180605&ll=43.6534817,-79.3839347&query=Hotel&radius=10000&limit=100'

In [78]:
#EXAMINE THE REQUESTS
results=requests.get(url).json()

In [79]:
venues= results['response']['venues']
df=json_normalize(venues)
df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ab2d511f964a5209b6c20e3,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1596008658,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ae61cf6f964a520caa421e3,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,Pantages Hotel & Spa,v-1596008658,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51d212c3498ebf27dc469bc9,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,Chelsea Hotel,v-1596008658,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4af96fbbf964a520c01122e3,1 King St W,CA,Toronto,Canada,at Yonge St.,686,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...","[{'label': 'display', 'lat': 43.6491395, 'lng'...",43.649139,-79.377876,NaN,M5H 1A1,ON,One King West Hotel & Residence,v-1596008658,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51e48697498eded9073c6c17,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,Crew Room Eaton Chelsea Hotel,v-1596008658,NaN


In [80]:
filtered_col=['name','categories']+[col for col in df.columns if col.startswith('location.')]+ ['id']
df2=df.loc[:,filtered_col]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
df2['categories'] = df2.apply(get_category_type, axis=1)

# clean columns
df2.columns = [col.split(".")[-1] for col in df2.columns]

df2.head()    


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
1,Pantages Hotel & Spa,Hotel,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,4ae61cf6f964a520caa421e3
2,Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,51d212c3498ebf27dc469bc9
3,One King West Hotel & Residence,Hotel,1 King St W,CA,Toronto,Canada,at Yonge St.,686,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...","[{'label': 'display', 'lat': 43.6491395, 'lng'...",43.649139,-79.377876,NaN,M5H 1A1,ON,4af96fbbf964a520c01122e3
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,51e48697498eded9073c6c17


In [81]:
df2_new= df2.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
df2_new.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
2,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
5,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
6,SoHo Metropolitan Hotel,Hotel,318 Wellington St W,43.644633,-79.392145,M5V 3T4,ON
7,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON
8,Le Germain Hotel Toronto Mercer,Hotel,30 Mercer St,43.645756,-79.390904,M5V 1H3,ON
9,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON


In [83]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_df= df2_new.loc[df2_new['categories'].isin(array)]
hotel_df.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
2,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
6,SoHo Metropolitan Hotel,Hotel,318 Wellington St W,43.644633,-79.392145,M5V 3T4,ON
7,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON
8,Le Germain Hotel Toronto Mercer,Hotel,30 Mercer St,43.645756,-79.390904,M5V 1H3,ON
9,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
10,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON


In [84]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_df.drop_duplicates(subset='name', keep="first")
df_hotels.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
2,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
6,SoHo Metropolitan Hotel,Hotel,318 Wellington St W,43.644633,-79.392145,M5V 3T4,ON
7,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON
8,Le Germain Hotel Toronto Mercer,Hotel,30 Mercer St,43.645756,-79.390904,M5V 1H3,ON
9,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
10,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON


In [86]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.38453,M5H 2M9,ON


# SEARCH FOR RESTAURANTS

In [87]:
search_query='Restaurants'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/search?&client_id=EOKYOPKM1KJJ301CDFXD1EZ1PBCZG0IQMJYZKANFDLHGKS5U&client_secret=H4BWIMNLNIJ14YONR3O0D2VWOJW0SXEE2MABSS5F2LDG5PW1&v=20180605&ll=43.6534817,-79.3839347&query=Restaurants&radius=10000&limit=100'

In [50]:
Rresults=requests.get(url).json()

In [88]:
venues=Rresults['response']['venues']
R_df=json_normalize(venues)
R_df.head(10)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b295e10f964a520ba9d24e3,14 Queen St W,CA,Toronto,Canada,NaN,313,"[14 Queen St W, Toronto ON M5H 3X4, Canada]","[{'label': 'display', 'lat': 43.65261436174172...",43.652614,-79.380231,M5H 3X4,ON,Richtree Natural Market Restaurants,v-1596008358,NaN
1,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4d14c68e380a8eec7b2185fa,"14 Duncan Street, Suite 203",CA,Toronto,Canada,NaN,831,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...","[{'label': 'display', 'lat': 43.64791, 'lng': ...",43.647910,-79.390810,M5H 3G2,ON,Imago Restaurants,v-1596008358,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4d8a7d78bc848cfabd63db2b,19 Baldwin Street,CA,Toronto,Canada,NaN,773,"[19 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65609618431269...",43.656096,-79.392839,NaN,ON,Valens Restaurants,v-1596008358,NaN
3,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,53a07ba3498ee8946e98a7de,552 Mt Pleasant,CA,Toronto,Canada,NaN,1132,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]","[{'label': 'display', 'lat': 43.64430171166487...",43.644302,-79.390002,M4S 2M6,ON,Marigold Indian Bistro | Indian Restaurants in...,v-1596008358,NaN
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,55eba3b8498e808c93025230,1155 Queen Street West,CA,Toronto,Canada,NaN,3547,"[1155 Queen Street West, Toronto ON M6J 1J4, C...","[{'label': 'display', 'lat': 43.64262834138462...",43.642628,-79.425337,M6J 1J4,ON,Restaurants Canada,v-1596008358,NaN
5,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,5ef05f2cc3f1e90008cc7133,326 Bloor St W,CA,Toronto,Canada,NaN,2170,"[326 Bloor St W, Toronto ON M5S 1W5, Canada]","[{'label': 'display', 'lat': 43.6669209, 'lng'...",43.666921,-79.403466,M5S 1W5,ON,Fresh Restaurants - Bloor,v-1596008358,582818058
6,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,50aff967e4b07ba2ce4a1c3c,"123 Queen Street West, Suite # C72",CA,Toronto,Canada,"At Sheraton Toronto Centre, Concourse",255,"[123 Queen Street West, Suite # C72 (At Sherat...","[{'label': 'display', 'lat': 43.65119366354807...",43.651194,-79.384234,M5H 3M9,ON,Subway,v-1596008358,NaN
7,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,4be9a0576295c9b61e738508,101 College St,CA,Toronto,Canada,NaN,805,"[101 College St, Toronto ON M5G 1L7, Canada]","[{'label': 'display', 'lat': 43.65987502486328...",43.659875,-79.388606,M5G 1L7,ON,Subway,v-1596008358,NaN
8,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,4b5f325ff964a52093ac29e3,"200 Wellington St West, Unit 340",CA,Toronto,Canada,NaN,894,"[200 Wellington St West, Unit 340, Toronto ON ...","[{'label': 'display', 'lat': 43.64595052717185...",43.645951,-79.387824,M5V 2C7,ON,Subway,v-1596008358,NaN
9,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,53850646498e3e40a5878024,"222 The Esplanade, Unit 1B",CA,Toronto,Canada,at Princess,1525,"[222 The Esplanade, Unit 1B (at Princess), Tor...","[{'label': 'display', 'lat': 43.64959405877831...",43.649594,-79.365779,M5A 4M8,ON,Subway,v-1596008358,NaN


In [89]:
R_filtered_col=['name','categories']+[col for col in R_df.columns if col.startswith('location.')]+ ['id']
R_df2=R_df.loc[:,R_filtered_col]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
R_df2['categories'] = R_df2.apply(get_category_type, axis=1)

# clean columns
R_df2.columns = [col.split(".")[-1] for col in R_df2.columns]

R_df2.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,CA,Toronto,Canada,NaN,313,"[14 Queen St W, Toronto ON M5H 3X4, Canada]","[{'label': 'display', 'lat': 43.65261436174172...",43.652614,-79.380231,M5H 3X4,ON,4b295e10f964a520ba9d24e3
1,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",CA,Toronto,Canada,NaN,831,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...","[{'label': 'display', 'lat': 43.64791, 'lng': ...",43.647910,-79.390810,M5H 3G2,ON,4d14c68e380a8eec7b2185fa
2,Valens Restaurants,Restaurant,19 Baldwin Street,CA,Toronto,Canada,NaN,773,"[19 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65609618431269...",43.656096,-79.392839,NaN,ON,4d8a7d78bc848cfabd63db2b
3,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,CA,Toronto,Canada,NaN,1132,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]","[{'label': 'display', 'lat': 43.64430171166487...",43.644302,-79.390002,M4S 2M6,ON,53a07ba3498ee8946e98a7de
4,Restaurants Canada,Office,1155 Queen Street West,CA,Toronto,Canada,NaN,3547,"[1155 Queen Street West, Toronto ON M6J 1J4, C...","[{'label': 'display', 'lat': 43.64262834138462...",43.642628,-79.425337,M6J 1J4,ON,55eba3b8498e808c93025230
5,Fresh Restaurants - Bloor,Restaurant,326 Bloor St W,CA,Toronto,Canada,NaN,2170,"[326 Bloor St W, Toronto ON M5S 1W5, Canada]","[{'label': 'display', 'lat': 43.6669209, 'lng'...",43.666921,-79.403466,M5S 1W5,ON,5ef05f2cc3f1e90008cc7133
6,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",CA,Toronto,Canada,"At Sheraton Toronto Centre, Concourse",255,"[123 Queen Street West, Suite # C72 (At Sherat...","[{'label': 'display', 'lat': 43.65119366354807...",43.651194,-79.384234,M5H 3M9,ON,50aff967e4b07ba2ce4a1c3c
7,Subway,Sandwich Place,101 College St,CA,Toronto,Canada,NaN,805,"[101 College St, Toronto ON M5G 1L7, Canada]","[{'label': 'display', 'lat': 43.65987502486328...",43.659875,-79.388606,M5G 1L7,ON,4be9a0576295c9b61e738508
8,Subway,Sandwich Place,"200 Wellington St West, Unit 340",CA,Toronto,Canada,NaN,894,"[200 Wellington St West, Unit 340, Toronto ON ...","[{'label': 'display', 'lat': 43.64595052717185...",43.645951,-79.387824,M5V 2C7,ON,4b5f325ff964a52093ac29e3
9,Subway,Sandwich Place,"222 The Esplanade, Unit 1B",CA,Toronto,Canada,at Princess,1525,"[222 The Esplanade, Unit 1B (at Princess), Tor...","[{'label': 'display', 'lat': 43.64959405877831...",43.649594,-79.365779,M5A 4M8,ON,53850646498e3e40a5878024


In [90]:
R_df2_new= R_df2.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
R_df2_new.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,M5H 3X4,ON
1,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",43.647910,-79.390810,M5H 3G2,ON
2,Valens Restaurants,Restaurant,19 Baldwin Street,43.656096,-79.392839,NaN,ON
3,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,43.644302,-79.390002,M4S 2M6,ON
4,Restaurants Canada,Office,1155 Queen Street West,43.642628,-79.425337,M6J 1J4,ON
5,Fresh Restaurants - Bloor,Restaurant,326 Bloor St W,43.666921,-79.403466,M5S 1W5,ON
6,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",43.651194,-79.384234,M5H 3M9,ON
7,Subway,Sandwich Place,101 College St,43.659875,-79.388606,M5G 1L7,ON
8,Subway,Sandwich Place,"200 Wellington St West, Unit 340",43.645951,-79.387824,M5V 2C7,ON
9,Subway,Sandwich Place,"222 The Esplanade, Unit 1B",43.649594,-79.365779,M5A 4M8,ON


In [92]:
df_Restaurant = R_df2_new.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,M5H 3X4,ON
1,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",43.647910,-79.390810,M5H 3G2,ON
3,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,43.644302,-79.390002,M4S 2M6,ON
4,Restaurants Canada,Office,1155 Queen Street West,43.642628,-79.425337,M6J 1J4,ON
5,Fresh Restaurants - Bloor,Restaurant,326 Bloor St W,43.666921,-79.403466,M5S 1W5,ON
6,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",43.651194,-79.384234,M5H 3M9,ON
7,Subway,Sandwich Place,101 College St,43.659875,-79.388606,M5G 1L7,ON
8,Subway,Sandwich Place,"200 Wellington St West, Unit 340",43.645951,-79.387824,M5V 2C7,ON
9,Subway,Sandwich Place,"222 The Esplanade, Unit 1B",43.649594,-79.365779,M5A 4M8,ON
11,Subway,Sandwich Place,"67 Regent Park Blvd, Unit 5",43.659959,-79.361179,M5A 3H6,ON


# SEARCH FOR CAFETERIA

In [58]:
search_query='Cafeteria'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/search?&client_id=EOKYOPKM1KJJ301CDFXD1EZ1PBCZG0IQMJYZKANFDLHGKS5U&client_secret=H4BWIMNLNIJ14YONR3O0D2VWOJW0SXEE2MABSS5F2LDG5PW1&v=20180605&ll=43.6534817,-79.3839347&query=Cafeteria&radius=10000&limit=100'

In [59]:
Cresults=requests.get(url).json()

In [93]:
venues=Cresults['response']['venues']
C_df=json_normalize(venues)
C_df.head(10)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4f9165bde4b04fef0087e4e2,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,448,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,Cafeteria,v-1596008479
1,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,528e99a211d262edc3708a6f,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,1246,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,Stay Cafeteria 慢走,v-1596008479
2,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4b57256bf964a520332828e3,Yonge,CA,Toronto,Canada,Dundas,387,"[Yonge (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65603785909391...",43.656038,-79.380672,NaN,ON,Sears Cafeteria,v-1596008479
3,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,53bd6dfe498e6c1d3bc9333e,NaN,CA,Toronto,Canada,NaN,505,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,Industrial Alliance Cafeteria,v-1596008479
4,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,524f1c8f498e983dde61ada0,89 Chestnut Street,CA,Toronto,Canada,NaN,584,"[89 Chestnut Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.648231, 'lng':...",43.648231,-79.384045,NaN,ON,Chestnut Tree Cafeteria,v-1596008479
5,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4ec2b68a8231a83de8c45191,100 McCaul St.,CA,Toronto,Canada,Second Floor,635,"[100 McCaul St. (Second Floor), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65278016892648...",43.652780,-79.391762,NaN,ON,OCAD Cafeteria,v-1596008479
6,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",False,556f826b498e07f469e6bf1f,NaN,CA,NaN,Canada,NaN,698,[Canada],"[{'label': 'display', 'lat': 43.65417861938476...",43.654179,-79.392555,NaN,NaN,Dears Cafeteria,v-1596008479
7,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4f187cbee4b0b34551e99f8d,350 Victoria St.,CA,Toronto,Canada,Gould St.,751,"[350 Victoria St. (Gould St.), Toronto ON, Can...","[{'label': 'display', 'lat': 43.65847495106884...",43.658475,-79.377653,NaN,ON,Ryerson Hub Cafeteria,v-1596008479
8,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4e7792bb45ddd43240ad83dc,555 University Avenue,CA,Toronto,Canada,NaN,808,"[555 University Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65758377430568...",43.657584,-79.375652,NaN,ON,SickKids Cafeteria,v-1596008479
9,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4e7641cdcc3fc1186ed1000a,131 Farnham Avenue,CA,Toronto,Canada,NaN,3471,"[131 Farnham Avenue, Toronto ON M4V 1H7, Canada]","[{'label': 'display', 'lat': 43.68300316722829...",43.683003,-79.397815,M4V 1H7,ON,De La Salle College Cafeteria,v-1596008479


In [94]:
C_filtered_col=['name','categories']+[col for col in C_df.columns if col.startswith('location.')]+ ['id']
C_df2=C_df.loc[:,C_filtered_col]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
C_df2['categories'] = C_df2.apply(get_category_type, axis=1)

# clean columns
C_df2.columns = [col.split(".")[-1] for col in C_df2.columns]

C_df2.head(10)    


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,448,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,1246,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,528e99a211d262edc3708a6f
2,Sears Cafeteria,Food Court,Yonge,CA,Toronto,Canada,Dundas,387,"[Yonge (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65603785909391...",43.656038,-79.380672,NaN,ON,4b57256bf964a520332828e3
3,Industrial Alliance Cafeteria,Cafeteria,NaN,CA,Toronto,Canada,NaN,505,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,53bd6dfe498e6c1d3bc9333e
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,CA,Toronto,Canada,NaN,584,"[89 Chestnut Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.648231, 'lng':...",43.648231,-79.384045,NaN,ON,524f1c8f498e983dde61ada0
5,OCAD Cafeteria,College Cafeteria,100 McCaul St.,CA,Toronto,Canada,Second Floor,635,"[100 McCaul St. (Second Floor), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65278016892648...",43.652780,-79.391762,NaN,ON,4ec2b68a8231a83de8c45191
6,Dears Cafeteria,Dessert Shop,NaN,CA,NaN,Canada,NaN,698,[Canada],"[{'label': 'display', 'lat': 43.65417861938476...",43.654179,-79.392555,NaN,NaN,556f826b498e07f469e6bf1f
7,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,CA,Toronto,Canada,Gould St.,751,"[350 Victoria St. (Gould St.), Toronto ON, Can...","[{'label': 'display', 'lat': 43.65847495106884...",43.658475,-79.377653,NaN,ON,4f187cbee4b0b34551e99f8d
8,SickKids Cafeteria,Food Court,555 University Avenue,CA,Toronto,Canada,NaN,808,"[555 University Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65758377430568...",43.657584,-79.375652,NaN,ON,4e7792bb45ddd43240ad83dc
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,CA,Toronto,Canada,NaN,3471,"[131 Farnham Avenue, Toronto ON M4V 1H7, Canada]","[{'label': 'display', 'lat': 43.68300316722829...",43.683003,-79.397815,M4V 1H7,ON,4e7641cdcc3fc1186ed1000a


In [95]:
C_df2_new= C_df2.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
C_df2_new.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON
5,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,NaN,ON
6,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
7,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,NaN,ON
8,SickKids Cafeteria,Food Court,555 University Avenue,43.657584,-79.375652,NaN,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON


In [96]:
df_Cafeteria = C_df2_new.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
19,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
21,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
24,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,M5S 1J5,ON
27,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON
31,Eurest Dishes Cafe,Food Court,100 Wynford Drive,43.727279,-79.332272,M3C 4B4,ON
33,Upstairs Cafe,Corporate Cafeteria,26 Prince Andrew Place,43.725383,-79.345481,M3C 2H4,ON


# SEARCH FOR SHOPPING STORE 

In [69]:
search_query='Shopping'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/search?&client_id=EOKYOPKM1KJJ301CDFXD1EZ1PBCZG0IQMJYZKANFDLHGKS5U&client_secret=H4BWIMNLNIJ14YONR3O0D2VWOJW0SXEE2MABSS5F2LDG5PW1&v=20180605&ll=43.6534817,-79.3839347&query=Shopping&radius=10000&limit=100'

In [70]:
Sresults=requests.get(url).json()

In [97]:
venues=Sresults['response']['venues']
S_df=json_normalize(venues)
S_df.head(10)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad4c062f964a520fcf720e3,280 Spadina Ave,CA,Toronto,Canada,at Dundas St W,1153,"[280 Spadina Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.65277367198862...",43.652774,-79.398222,NaN,M5T 3A5,ON,Dragon City Shopping Mall 龍城,v-1596008583,NaN
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad4c062f964a52002f820e3,3401 Dufferin Street,CA,Toronto,Canada,NaN,9742,"[3401 Dufferin Street, Toronto ON M6A 2T9, Can...","[{'label': 'display', 'lat': 43.72548184266099...",43.725482,-79.452736,NaN,M6A 2T9,ON,Yorkdale Shopping Centre,v-1596008583,NaN
2,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,5840cafe06f1a34af80cc609,176 Yonge Street,CA,Toronto,Canada,Queen Street West,424,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,Saks Fifth Avenue Club - Personal Shopping,v-1596008583,NaN
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,5044fe32e4b0c55b95cc2efd,291 York Mills Rd.,CA,North York,Canada,at Bayview Ave.,10491,"[291 York Mills Rd. (at Bayview Ave.), North Y...","[{'label': 'display', 'lat': 43.74773098016195...",43.747731,-79.384724,NaN,NaN,ON,York Mills Shopping Centre,v-1596008583,NaN
4,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad77a12f964a520260b21e3,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,287,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654540,-79.380677,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,CF Toronto Eaton Centre,v-1596008583,NaN
5,"[{'id': '52e81612bcbc57f1066b7a3d', 'name': 'A...",False,5842818b7d0f6d56903c7095,993 Bloor Street West,CA,Toronto,Canada,Dufferin,3780,"[993 Bloor Street West (Dufferin), Toronto Sel...","[{'label': 'display', 'lat': 43.66096414412844...",43.660964,-79.429725,NaN,M6H 1M1,Select State,"QR Code Shopping, Best of the Best Selling!",v-1596008583,NaN
6,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b9e86dff964a520e6eb36e3,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,741,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,TD Centre Shopping Concourse,v-1596008583,NaN
7,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,4c1843416a21c9b674dec897,250 Donlands Ave,CA,Toronto,Canada,at Memorial Park Ave,5279,"[250 Donlands Ave (at Memorial Park Ave), Toro...","[{'label': 'display', 'lat': 43.689219, 'lng':...",43.689219,-79.340836,NaN,M4J,ON,Shopping Basket,v-1596008583,NaN
8,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,4ef4758261af302a97b71465,NaN,CA,Toronto,Canada,NaN,2619,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64274007508189...",43.642740,-79.412871,NaN,NaN,ON,Last Minute Shopping,v-1596008583,NaN
9,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,516045ace4b09c0b6099b5f7,NaN,CA,Toronto,Canada,NaN,11601,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.74263, 'lng': ...",43.742630,-79.309260,NaN,NaN,ON,Victoria terrace shopping centre,v-1596008583,NaN


In [98]:
S_filtered_col=['name','categories']+[col for col in S_df.columns if col.startswith('location.')]+ ['id']
S_df2=S_df.loc[:,S_filtered_col]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
S_df2['categories'] = S_df2.apply(get_category_type, axis=1)

# clean columns
S_df2.columns = [col.split(".")[-1] for col in S_df2.columns]

S_df2.head(10)


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,CA,Toronto,Canada,at Dundas St W,1153,"[280 Spadina Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.65277367198862...",43.652774,-79.398222,NaN,M5T 3A5,ON,4ad4c062f964a520fcf720e3
1,Yorkdale Shopping Centre,Shopping Mall,3401 Dufferin Street,CA,Toronto,Canada,NaN,9742,"[3401 Dufferin Street, Toronto ON M6A 2T9, Can...","[{'label': 'display', 'lat': 43.72548184266099...",43.725482,-79.452736,NaN,M6A 2T9,ON,4ad4c062f964a52002f820e3
2,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,CA,Toronto,Canada,Queen Street West,424,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,5840cafe06f1a34af80cc609
3,York Mills Shopping Centre,Shopping Mall,291 York Mills Rd.,CA,North York,Canada,at Bayview Ave.,10491,"[291 York Mills Rd. (at Bayview Ave.), North Y...","[{'label': 'display', 'lat': 43.74773098016195...",43.747731,-79.384724,NaN,NaN,ON,5044fe32e4b0c55b95cc2efd
4,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,287,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654540,-79.380677,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3
5,"QR Code Shopping, Best of the Best Selling!",Advertising Agency,993 Bloor Street West,CA,Toronto,Canada,Dufferin,3780,"[993 Bloor Street West (Dufferin), Toronto Sel...","[{'label': 'display', 'lat': 43.66096414412844...",43.660964,-79.429725,NaN,M6H 1M1,Select State,5842818b7d0f6d56903c7095
6,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,741,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,4b9e86dff964a520e6eb36e3
7,Shopping Basket,Convenience Store,250 Donlands Ave,CA,Toronto,Canada,at Memorial Park Ave,5279,"[250 Donlands Ave (at Memorial Park Ave), Toro...","[{'label': 'display', 'lat': 43.689219, 'lng':...",43.689219,-79.340836,NaN,M4J,ON,4c1843416a21c9b674dec897
8,Last Minute Shopping,Department Store,NaN,CA,Toronto,Canada,NaN,2619,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64274007508189...",43.642740,-79.412871,NaN,NaN,ON,4ef4758261af302a97b71465
9,Victoria terrace shopping centre,Shopping Mall,NaN,CA,Toronto,Canada,NaN,11601,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.74263, 'lng': ...",43.742630,-79.309260,NaN,NaN,ON,516045ace4b09c0b6099b5f7


In [75]:
S_df2_new= S_df2.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
S_df2_new.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,Yorkdale Shopping Centre,Shopping Mall,3401 Dufferin Street,43.725482,-79.452736,M6A 2T9,ON
2,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
3,York Mills Shopping Centre,Shopping Mall,291 York Mills Rd.,43.747731,-79.384724,NaN,ON
4,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
5,"QR Code Shopping, Best of the Best Selling!",Advertising Agency,993 Bloor Street West,43.660964,-79.429725,M6H 1M1,Select State
6,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
7,Shopping Basket,Convenience Store,250 Donlands Ave,43.689219,-79.340836,M4J,ON
8,Last Minute Shopping,Department Store,NaN,43.642740,-79.412871,NaN,ON
9,Victoria terrace shopping centre,Shopping Mall,NaN,43.742630,-79.309260,NaN,ON


In [101]:
df_Shopping = S_df2_new.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Shopping.head(10)

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,Yorkdale Shopping Centre,Shopping Mall,3401 Dufferin Street,43.725482,-79.452736,M6A 2T9,ON
2,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
4,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
5,"QR Code Shopping, Best of the Best Selling!",Advertising Agency,993 Bloor Street West,43.660964,-79.429725,M6H 1M1,Select State
6,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
7,Shopping Basket,Convenience Store,250 Donlands Ave,43.689219,-79.340836,M4J,ON
11,Upper Oakville Shopping Centre Head Office,Building,21 Booth Avenue Suite 200,43.654294,-79.340546,M4M 2M3,ON
12,Riverdale Shopping Centre,Shopping Plaza,449 Carlaw Ave,43.668694,-79.342070,M4K 3J1,ON
13,Galleria Shopping Centre,Shopping Mall,1245 Dupont St,43.667592,-79.442053,M6H 2A6,ON


# CLUSTER TOGETHER HOTELS, RESTAURANTS, CAFETERIA & SHOPPING STORES

CLUSTERING

In [106]:
# create dataframe of hotels, shopping stores and Cafeteria
neighbourhood_df = pd.concat([df_hotel, df_Restaurant, df_Cafeteria, df_Shopping], ignore_index=True)
neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,M5H 3X4,ON
2,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",43.647910,-79.390810,M5H 3G2,ON
3,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,43.644302,-79.390002,M4S 2M6,ON
4,Restaurants Canada,Office,1155 Queen Street West,43.642628,-79.425337,M6J 1J4,ON
5,Fresh Restaurants - Bloor,Restaurant,326 Bloor St W,43.666921,-79.403466,M5S 1W5,ON
6,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",43.651194,-79.384234,M5H 3M9,ON
7,Subway,Sandwich Place,101 College St,43.659875,-79.388606,M5G 1L7,ON
8,Subway,Sandwich Place,"200 Wellington St West, Unit 340",43.645951,-79.387824,M5V 2C7,ON
9,Subway,Sandwich Place,"222 The Esplanade, Unit 1B",43.649594,-79.365779,M5A 4M8,ON


In [107]:
Tornoto_map=folium.Map(location=[latitude,longitude],zoom_start=14)
for lat, lng, name, categories, address in zip(neighbourhood_df['lat'], neighbourhood_df['lng'], neighbourhood_df['name'], neighbourhood_df['categories'],neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Tornoto_map)  

Tornoto_map